The ETL process is as follows:
1. Import the database tables as dataframes
2. Use pandas dataframe operations to clean the dataframe
3. Extract dimension values
4. Load it to the data warehouse

Standard Protocols will be used to clean data
1. Checking for incorrect data types
2. Check for dupliucate values
3. Check for multiple representations
4. Check for missing and default values
5. Check for inconsistent format

In [4]:
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy import URL
import numpy as np
from enum import Enum
import sqlalchemy

In [5]:
# creating engine. MySQL will be used
url_object = URL.create(
    drivername='mysql',
    username='root',
    password='Data_101',
    host='localhost',
    port=3306,
    database='seriousmd'
)
MySqlEngine = create_engine(url_object)

In [6]:
# creating engine for data warehouse
url_object = URL.create(
    drivername='mysql',
    username='root',
    password='Data_101',
    host='localhost',
    port=3306,
    database='mdwarehouse'
)
MySqlEngineDW = create_engine(url_object)

In [7]:
# importing the appointments table to a dataframe for
seriousmd_conn = MySqlEngine.connect()

appointments_df = pd.read_sql(
    sql="""SELECT * FROM appointments""",
    con=seriousmd_conn,
    parse_dates={
        'TimeQueued':'%Y-%m-%d %H:%M:%S',
        'QueueDate':'%Y-%m-%d %H:%M:%S',
        'StartTime':'%Y-%m-%d %H:%M:%S',
        'EndTime':'%Y-%m-%d %H:%M:%S'
    }
)
appointments_df.info()
appointments_df.head()

seriousmd_conn.close()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 522346 entries, 0 to 522345
Data columns (total 11 columns):
 #   Column      Non-Null Count   Dtype         
---  ------      --------------   -----         
 0   pxid        522346 non-null  object        
 1   clinicid    522346 non-null  object        
 2   doctorid    522346 non-null  object        
 3   apptid      522346 non-null  object        
 4   status      522346 non-null  object        
 5   TimeQueued  522346 non-null  datetime64[ns]
 6   QueueDate   522346 non-null  datetime64[ns]
 7   StartTime   352714 non-null  datetime64[ns]
 8   EndTime     90502 non-null   datetime64[ns]
 9   type        522346 non-null  object        
 10  Virtual     522346 non-null  object        
dtypes: datetime64[ns](4), object(7)
memory usage: 43.8+ MB


**Appointments data cleaning**

In [ ]:
# all data types are good

Checking for duplicated rows

In [8]:
# checking for duplicated rows
appointments_df[appointments_df.duplicated()]

,pxid,clinicid,doctorid,apptid,status,TimeQueued,QueueDate,StartTime,EndTime,type,Virtual
1,FE4A5D5A20EC492D2FC691F126A568AB,3B8D83483189887A2F1A39D690463A8F,ACB3A881C7CE9ABCAE0CE8C99C86A906,04BC9218E072BEEFEBEE9C97B78A35C9,Queued,2020-06-22 05:11:38,2020-07-02 16:00:00,2020-07-02 17:00:38,2020-07-02 17:15:38,Consultation,
3,E6BC8E80685AA4A239617F196F12A0C3,3B8D83483189887A2F1A39D690463A8F,ACB3A881C7CE9ABCAE0CE8C99C86A906,F2335AAFB7EE079C7E7306591057C0C9,Queued,2020-06-26 04:51:16,2020-07-12 16:00:00,2020-07-12 17:00:16,2020-07-12 17:15:16,Consultation,
5,E430B3D3AEED550612FFDBD0F2F94112,3B8D83483189887A2F1A39D690463A8F,ACB3A881C7CE9ABCAE0CE8C99C86A906,B2C83D6DE89A88C5CD9EA1FD5FC00602,Queued,2020-07-13 07:00:50,2020-07-19 16:00:00,2020-07-20 05:00:50,2020-07-20 05:15:50,Consultation,
7,E6BC8E80685AA4A239617F196F12A0C3,3B8D83483189887A2F1A39D690463A8F,ACB3A881C7CE9ABCAE0CE8C99C86A906,DA4023B5A8C3F1BD540EA82552F21134,Queued,2020-07-13 06:07:14,2020-07-28 16:00:00,2020-07-29 05:00:13,2020-07-29 05:15:13,Consultation,
9,41E3F930274A7704305EE197F3434877,3B8D83483189887A2F1A39D690463A8F,ACB3A881C7CE9ABCAE0CE8C99C86A906,C805F7992F18E33AB85CEBD572680943,Queued,2020-06-26 05:54:35,2020-09-27 16:00:00,2020-09-27 17:00:35,2020-09-27 17:15:35,Consultation,
...,...,...,...,...,...,...,...,...,...,...,...
439189,63AA10BF9075360680D37E621B1B6BA5,321BBE1B2890962B06574A806B693EE5,3464CE4186536A9855A8A7967B121B3E,E6EA32383B6007189D31592604CEB0F7,Complete,2023-09-30 07:59:21,2023-09-29 16:00:00,2023-09-30 08:00:00,2023-09-30 08:30:00,Consultation,False
439191,63AA10BF9075360680D37E621B1B6BA5,321BBE1B2890962B06574A806B693EE5,3464CE4186536A9855A8A7967B121B3E,D69D6F448CE9C6A58905D36776B333B1,Complete,2023-10-15 09:35:38,2023-10-14 16:00:00,2023-10-15 09:00:00,2023-10-15 09:15:00,Consultation,False
439193,63AA10BF9075360680D37E621B1B6BA5,321BBE1B2890962B06574A806B693EE5,3464CE4186536A9855A8A7967B121B3E,8D17DCFF68FFCAEC2574D8D16558B6D1,Complete,2023-10-28 07:27:26,2023-10-27 16:00:00,2023-10-28 07:30:00,2023-10-28 08:00:00,Consultation,False
439195,63AA10BF9075360680D37E621B1B6BA5,321BBE1B2890962B06574A806B693EE5,3464CE4186536A9855A8A7967B121B3E,0827C6F8480B4B8ACB89B27E592EEF81,Complete,2024-01-11 07:42:26,2024-01-10 16:00:00,2024-01-11 07:00:00,2024-01-11 07:30:00,Consultation,False


In [9]:
# getting the index of duplicated values
duplicated_index = appointments_df[appointments_df.duplicated()].index

In [10]:
# dropping the duplicate rows
appointments_df = appointments_df.drop(index=duplicated_index)
appointments_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 320140 entries, 0 to 522345
Data columns (total 11 columns):
 #   Column      Non-Null Count   Dtype         
---  ------      --------------   -----         
 0   pxid        320140 non-null  object        
 1   clinicid    320140 non-null  object        
 2   doctorid    320140 non-null  object        
 3   apptid      320140 non-null  object        
 4   status      320140 non-null  object        
 5   TimeQueued  320140 non-null  datetime64[ns]
 6   QueueDate   320140 non-null  datetime64[ns]
 7   StartTime   203608 non-null  datetime64[ns]
 8   EndTime     60152 non-null   datetime64[ns]
 9   type        320140 non-null  object        
 10  Virtual     320140 non-null  object        
dtypes: datetime64[ns](4), object(7)
memory usage: 29.3+ MB


In [11]:
# resetting the index
appointments_df.reset_index(drop=True, inplace=True)
appointments_df.head()

,pxid,clinicid,doctorid,apptid,status,TimeQueued,QueueDate,StartTime,EndTime,type,Virtual
0,FE4A5D5A20EC492D2FC691F126A568AB,3B8D83483189887A2F1A39D690463A8F,ACB3A881C7CE9ABCAE0CE8C99C86A906,04BC9218E072BEEFEBEE9C97B78A35C9,Queued,2020-06-22 05:11:38,2020-07-02 16:00:00,2020-07-02 17:00:38,2020-07-02 17:15:38,Consultation,
1,E6BC8E80685AA4A239617F196F12A0C3,3B8D83483189887A2F1A39D690463A8F,ACB3A881C7CE9ABCAE0CE8C99C86A906,F2335AAFB7EE079C7E7306591057C0C9,Queued,2020-06-26 04:51:16,2020-07-12 16:00:00,2020-07-12 17:00:16,2020-07-12 17:15:16,Consultation,
2,E430B3D3AEED550612FFDBD0F2F94112,3B8D83483189887A2F1A39D690463A8F,ACB3A881C7CE9ABCAE0CE8C99C86A906,B2C83D6DE89A88C5CD9EA1FD5FC00602,Queued,2020-07-13 07:00:50,2020-07-19 16:00:00,2020-07-20 05:00:50,2020-07-20 05:15:50,Consultation,
3,E6BC8E80685AA4A239617F196F12A0C3,3B8D83483189887A2F1A39D690463A8F,ACB3A881C7CE9ABCAE0CE8C99C86A906,DA4023B5A8C3F1BD540EA82552F21134,Queued,2020-07-13 06:07:14,2020-07-28 16:00:00,2020-07-29 05:00:13,2020-07-29 05:15:13,Consultation,
4,41E3F930274A7704305EE197F3434877,3B8D83483189887A2F1A39D690463A8F,ACB3A881C7CE9ABCAE0CE8C99C86A906,C805F7992F18E33AB85CEBD572680943,Queued,2020-06-26 05:54:35,2020-09-27 16:00:00,2020-09-27 17:00:35,2020-09-27 17:15:35,Consultation,


Checking for multiple representations

In [12]:
appointments_df['status'].unique()

array(['Queued', 'Complete', 'Serving', 'Cancel', 'NoShow', 'Skip'],
      dtype=object)

In [13]:
appointments_df['type'].unique()

array(['Consultation', 'Inpatient'], dtype=object)

In [14]:
appointments_df['Virtual'].unique()

array(['', 'True', 'False'], dtype=object)

Check for missing and default values

In [15]:
appointments_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 320140 entries, 0 to 320139
Data columns (total 11 columns):
 #   Column      Non-Null Count   Dtype         
---  ------      --------------   -----         
 0   pxid        320140 non-null  object        
 1   clinicid    320140 non-null  object        
 2   doctorid    320140 non-null  object        
 3   apptid      320140 non-null  object        
 4   status      320140 non-null  object        
 5   TimeQueued  320140 non-null  datetime64[ns]
 6   QueueDate   320140 non-null  datetime64[ns]
 7   StartTime   203608 non-null  datetime64[ns]
 8   EndTime     60152 non-null   datetime64[ns]
 9   type        320140 non-null  object        
 10  Virtual     320140 non-null  object        
dtypes: datetime64[ns](4), object(7)
memory usage: 26.9+ MB


In [19]:
# Virtual has missing values, we must set it to nan
appointments_df['Virtual'].replace(to_replace=r'^\s*$', value=np.nan, regex=True, inplace=True)

# StartTime and EndTime also has missing values, but they are already nan
appointments_df[appointments_df['StartTime'].isna()]

C:\Users\Francis\AppData\Local\Temp\ipykernel_3812\17297632.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  appointments_df['Virtual'].replace(to_replace=r'^\s*$', value=np.nan, regex=True, inplace=True)


,pxid,clinicid,doctorid,apptid,status,TimeQueued,QueueDate,StartTime,EndTime,type,Virtual
602,A4B7F2DEFAB485D3E823EC2E2887CE8B,287ABB19DA8AADBD118443E92685853E,ACB3A881C7CE9ABCAE0CE8C99C86A906,99D860AB38428CB8A5D288083E360FE1,Complete,2023-10-24 02:10:09,2023-10-23 16:00:00,NaT,NaT,Consultation,True
1663,B61BB5CAA06916F0E7F095678F1160D1,BAE3529F0210CC5A603AC708BB5E7797,ACB3A881C7CE9ABCAE0CE8C99C86A906,C088134FD423E91DDD05F6F07D0F9AA1,Cancel,2022-08-18 05:49:29,2022-08-17 16:00:00,NaT,NaT,Consultation,False
1729,CEC712367A9D73263198E89D2740D665,40ACE1BABB133E1EA17F09932DD2E508,30A250583BC20CF070AB6C9189508FCC,B040FF3DF6C113BA3CCB67CAECD231A3,Complete,2021-09-03 06:09:09,2021-09-02 16:00:00,NaT,NaT,Consultation,NaN
1771,8D04D68B3704C23A6F94DEA573376BDC,40ACE1BABB133E1EA17F09932DD2E508,30A250583BC20CF070AB6C9189508FCC,925FB021CB5D93C0E5E427A1BAED89E7,Complete,2021-11-05 05:18:50,2021-11-04 16:00:00,NaT,NaT,Consultation,True
1784,7413080D5B545CCC89F893BE2C234430,40ACE1BABB133E1EA17F09932DD2E508,30A250583BC20CF070AB6C9189508FCC,411A4CCB538A7AE40FD06DE85141DE04,Complete,2022-01-04 03:13:46,2022-01-03 16:00:00,NaT,NaT,Consultation,False
...,...,...,...,...,...,...,...,...,...,...,...
319717,9D18F13E9682D17E9927B05AB66D1B53,B5F73D2F8C5E4AA26041EFFE5FBDF930,40F4DA34BBE180214C23B9E55DA4F772,54EAA1A67288C240F6F90AF8519A64D1,Queued,2022-01-25 09:03:19,2022-02-07 16:00:00,NaT,NaT,Consultation,True
319719,804BD0FC23BBAE33A9B3CDD4B0316407,B5F73D2F8C5E4AA26041EFFE5FBDF930,40F4DA34BBE180214C23B9E55DA4F772,27DE452F130F75364DA35A1118DEDA90,Complete,2022-03-26 05:27:30,2022-03-25 16:00:00,NaT,NaT,Consultation,True
319723,180728A1AB4FA0B7E04FD98330D7E176,B5F73D2F8C5E4AA26041EFFE5FBDF930,40F4DA34BBE180214C23B9E55DA4F772,FE98FA7C8F8E514915B48FD50B61FDF3,Complete,2022-05-16 09:32:24,2022-05-15 16:00:00,NaT,NaT,Consultation,True
319730,9D895E7EE02B955F12031933D3919F39,B5F73D2F8C5E4AA26041EFFE5FBDF930,40F4DA34BBE180214C23B9E55DA4F772,D6273DDDE4C6B001F69378567C414254,Queued,2024-01-13 02:35:34,2024-03-07 16:00:00,NaT,NaT,Consultation,True


In [20]:
appointments_df[appointments_df['EndTime'].isna()]

# I will do nothing as there is nothing that I can do

,pxid,clinicid,doctorid,apptid,status,TimeQueued,QueueDate,StartTime,EndTime,type,Virtual
602,A4B7F2DEFAB485D3E823EC2E2887CE8B,287ABB19DA8AADBD118443E92685853E,ACB3A881C7CE9ABCAE0CE8C99C86A906,99D860AB38428CB8A5D288083E360FE1,Complete,2023-10-24 02:10:09,2023-10-23 16:00:00,NaT,NaT,Consultation,True
1663,B61BB5CAA06916F0E7F095678F1160D1,BAE3529F0210CC5A603AC708BB5E7797,ACB3A881C7CE9ABCAE0CE8C99C86A906,C088134FD423E91DDD05F6F07D0F9AA1,Cancel,2022-08-18 05:49:29,2022-08-17 16:00:00,NaT,NaT,Consultation,False
1729,CEC712367A9D73263198E89D2740D665,40ACE1BABB133E1EA17F09932DD2E508,30A250583BC20CF070AB6C9189508FCC,B040FF3DF6C113BA3CCB67CAECD231A3,Complete,2021-09-03 06:09:09,2021-09-02 16:00:00,NaT,NaT,Consultation,NaN
1771,8D04D68B3704C23A6F94DEA573376BDC,40ACE1BABB133E1EA17F09932DD2E508,30A250583BC20CF070AB6C9189508FCC,925FB021CB5D93C0E5E427A1BAED89E7,Complete,2021-11-05 05:18:50,2021-11-04 16:00:00,NaT,NaT,Consultation,True
1784,7413080D5B545CCC89F893BE2C234430,40ACE1BABB133E1EA17F09932DD2E508,30A250583BC20CF070AB6C9189508FCC,411A4CCB538A7AE40FD06DE85141DE04,Complete,2022-01-04 03:13:46,2022-01-03 16:00:00,NaT,NaT,Consultation,False
...,...,...,...,...,...,...,...,...,...,...,...
319723,180728A1AB4FA0B7E04FD98330D7E176,B5F73D2F8C5E4AA26041EFFE5FBDF930,40F4DA34BBE180214C23B9E55DA4F772,FE98FA7C8F8E514915B48FD50B61FDF3,Complete,2022-05-16 09:32:24,2022-05-15 16:00:00,NaT,NaT,Consultation,True
319730,9D895E7EE02B955F12031933D3919F39,B5F73D2F8C5E4AA26041EFFE5FBDF930,40F4DA34BBE180214C23B9E55DA4F772,D6273DDDE4C6B001F69378567C414254,Queued,2024-01-13 02:35:34,2024-03-07 16:00:00,NaT,NaT,Consultation,True
319731,DA6746BB79186CDBA25FB834A37E2CCF,4776DA547036453F71AB92AAD7C727BD,F7F07E7DAB09533BC71247A5B29A7373,7125DEE97DCE77AAE68AED4C6C36B8A2,Queued,2020-05-22 03:58:06,2020-05-21 16:00:00,2020-05-22 03:57:05,NaT,Inpatient,NaN
320050,737E1BEBACD52D564B5E091B18FD4F77,4776DA547036453F71AB92AAD7C727BD,F7F07E7DAB09533BC71247A5B29A7373,99FED9BA404058DEA8180CE7F0D8E63A,NoShow,2022-07-16 01:54:00,2022-07-22 16:00:00,NaT,NaT,Consultation,False


Checking for inconsistent format

In [21]:
list_of_valid_format_starttime = appointments_df['StartTime'].astype(str).str.match(r'^\d{4}-\d{2}-\d{2} \d{2}:\d{2}:\d{2}$')
appointments_df[~list_of_valid_format_starttime]['StartTime'].unique()

<DatetimeArray>
['NaT']
Length: 1, dtype: datetime64[ns]

In [22]:
list_of_valid_format_endtime = appointments_df['EndTime'].astype(str).str.match(r'^\d{4}-\d{2}-\d{2} \d{2}:\d{2}:\d{2}$')
appointments_df[~list_of_valid_format_endtime]['EndTime'].unique()

# Besides NaT, there are no other strings that violates the yyyy-MM-dd HH-mm-ss format

<DatetimeArray>
['NaT']
Length: 1, dtype: datetime64[ns]

Since the appointments will be the fact table, this will be loaded directly to the date warehouse.

Renaming columns to follow database model

In [23]:
appointments_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 320140 entries, 0 to 320139
Data columns (total 11 columns):
 #   Column      Non-Null Count   Dtype         
---  ------      --------------   -----         
 0   pxid        320140 non-null  object        
 1   clinicid    320140 non-null  object        
 2   doctorid    320140 non-null  object        
 3   apptid      320140 non-null  object        
 4   status      320140 non-null  object        
 5   TimeQueued  320140 non-null  datetime64[ns]
 6   QueueDate   320140 non-null  datetime64[ns]
 7   StartTime   203608 non-null  datetime64[ns]
 8   EndTime     60152 non-null   datetime64[ns]
 9   type        320140 non-null  object        
 10  Virtual     130004 non-null  object        
dtypes: datetime64[ns](4), object(7)
memory usage: 26.9+ MB


In [24]:
appointments_df.rename(
    columns={
        'pxid': 'pxID',
        'clinicid': 'clinicID',
        'doctorid': 'doctorID',
        'apptid': 'apptID',
        'TimeQueued': 'timequeued',
        'QueueDate': 'queuedate',
        'StartTime': 'starttime',
        'EndTime': 'endtime',
        'Virtual': 'virtual'
    },
    copy=False,
    inplace=True,
    errors='raise'
)

In [25]:
appointments_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 320140 entries, 0 to 320139
Data columns (total 11 columns):
 #   Column      Non-Null Count   Dtype         
---  ------      --------------   -----         
 0   pxID        320140 non-null  object        
 1   clinicID    320140 non-null  object        
 2   doctorID    320140 non-null  object        
 3   apptID      320140 non-null  object        
 4   status      320140 non-null  object        
 5   timequeued  320140 non-null  datetime64[ns]
 6   queuedate   320140 non-null  datetime64[ns]
 7   starttime   203608 non-null  datetime64[ns]
 8   endtime     60152 non-null   datetime64[ns]
 9   type        320140 non-null  object        
 10  virtual     130004 non-null  object        
dtypes: datetime64[ns](4), object(7)
memory usage: 26.9+ MB


In [26]:
# Prepareing data type
class TypeEnum(Enum):
    Inpatient = 'Inpatient'
    Consultation = 'Consultation'

class StatusEnum(Enum):
    Queued = 'Queued'
    Complete = 'Complete'
    Serving = 'Serving'
    Cancel = 'Cancel'
    NoShow = 'NoShow'
    Skip = 'Skip'

In [27]:
# transforming true or false to python true or false
appointments_df['virtual'] = appointments_df['virtual'].map({'True': True, 'False': False}, na_action='ignore')

In [28]:
# sending dataframe to sql table
mdwarehouse_conn = MySqlEngineDW.connect()

rows_affected = appointments_df.head(100).to_sql(
    name='fact_appointment',
    con=mdwarehouse_conn,
    if_exists='append',
    index=False,
    dtype={
        'pxID': sqlalchemy.types.String(32),
        'clinicID': sqlalchemy.types.String(32),
        'doctorID': sqlalchemy.types.String(32),
        'apptID': sqlalchemy.types.String(32),
        'status':sqlalchemy.types.Enum(StatusEnum),
        'timequeued': sqlalchemy.types.DateTime,
        'queuedate': sqlalchemy.types.DateTime,
        'starttime': sqlalchemy.types.DateTime,
        'endtime': sqlalchemy.types.DateTime,
        'type': sqlalchemy.types.Enum(TypeEnum),
        'virtual': sqlalchemy.types.Boolean
    },
    chunksize=5000,
    method='multi'
)

mdwarehouse_conn.close()

IntegrityError: (MySQLdb.IntegrityError) (1452, 'Cannot add or update a child row: a foreign key constraint fails (`mdwarehouse`.`fact_appointment`, CONSTRAINT `clinicID_FK` FOREIGN KEY (`clinicID`) REFERENCES `dim_clinic` (`clinicID`))')
[SQL: INSERT INTO fact_appointment (`pxID`, `clinicID`, `doctorID`, `apptID`, status, timequeued, queuedate, starttime, endtime, type, `virtual`) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)]
[parameters: [('FE4A5D5A20EC492D2FC691F126A568AB', '3B8D83483189887A2F1A39D690463A8F', 'ACB3A881C7CE9ABCAE0CE8C99C86A906', '04BC9218E072BEEFEBEE9C97B78A35C9', 'Queued', datetime.datetime(2020, 6, 22, 5, 11, 38), datetime.datetime(2020, 7, 2, 16, 0), datetime.datetime(2020, 7, 2, 17, 0, 38), datetime.datetime(2020, 7, 2, 17, 15, 38), 'Consultation', None), ('E6BC8E80685AA4A239617F196F12A0C3', '3B8D83483189887A2F1A39D690463A8F', 'ACB3A881C7CE9ABCAE0CE8C99C86A906', 'F2335AAFB7EE079C7E7306591057C0C9', 'Queued', datetime.datetime(2020, 6, 26, 4, 51, 16), dateti

In [29]:
appointments_df.head(100)

,pxID,clinicID,doctorID,apptID,status,timequeued,queuedate,starttime,endtime,type,virtual
0,FE4A5D5A20EC492D2FC691F126A568AB,3B8D83483189887A2F1A39D690463A8F,ACB3A881C7CE9ABCAE0CE8C99C86A906,04BC9218E072BEEFEBEE9C97B78A35C9,Queued,2020-06-22 05:11:38,2020-07-02 16:00:00,2020-07-02 17:00:38,2020-07-02 17:15:38,Consultation,NaN
1,E6BC8E80685AA4A239617F196F12A0C3,3B8D83483189887A2F1A39D690463A8F,ACB3A881C7CE9ABCAE0CE8C99C86A906,F2335AAFB7EE079C7E7306591057C0C9,Queued,2020-06-26 04:51:16,2020-07-12 16:00:00,2020-07-12 17:00:16,2020-07-12 17:15:16,Consultation,NaN
2,E430B3D3AEED550612FFDBD0F2F94112,3B8D83483189887A2F1A39D690463A8F,ACB3A881C7CE9ABCAE0CE8C99C86A906,B2C83D6DE89A88C5CD9EA1FD5FC00602,Queued,2020-07-13 07:00:50,2020-07-19 16:00:00,2020-07-20 05:00:50,2020-07-20 05:15:50,Consultation,NaN
3,E6BC8E80685AA4A239617F196F12A0C3,3B8D83483189887A2F1A39D690463A8F,ACB3A881C7CE9ABCAE0CE8C99C86A906,DA4023B5A8C3F1BD540EA82552F21134,Queued,2020-07-13 06:07:14,2020-07-28 16:00:00,2020-07-29 05:00:13,2020-07-29 05:15:13,Consultation,NaN
4,41E3F930274A7704305EE197F3434877,3B8D83483189887A2F1A39D690463A8F,ACB3A881C7CE9ABCAE0CE8C99C86A906,C805F7992F18E33AB85CEBD572680943,Queued,2020-06-26 05:54:35,2020-09-27 16:00:00,2020-09-27 17:00:35,2020-09-27 17:15:35,Consultation,NaN
...,...,...,...,...,...,...,...,...,...,...,...
95,0173144A9589D9B631D1213CB68CD1F3,287ABB19DA8AADBD118443E92685853E,ACB3A881C7CE9ABCAE0CE8C99C86A906,24BBEF21D8B8E1F63BF9D95D22C52854,Complete,2020-11-19 04:26:14,2020-11-19 16:00:00,2020-11-20 01:30:00,2020-11-20 01:45:00,Consultation,False
96,A4B7F2DEFAB485D3E823EC2E2887CE8B,287ABB19DA8AADBD118443E92685853E,ACB3A881C7CE9ABCAE0CE8C99C86A906,9ECD6AD001663BFF8259BF04932B287E,Complete,2020-11-23 23:37:41,2020-11-23 16:00:00,2020-11-24 03:15:00,2020-11-24 03:30:00,Consultation,True
97,17F29BDC62FF6C1E3EADA946F41BD78C,287ABB19DA8AADBD118443E92685853E,ACB3A881C7CE9ABCAE0CE8C99C86A906,1F61DE829ECDBAA4E09FB099781C47E8,Complete,2020-11-20 02:25:35,2020-11-23 16:00:00,2020-11-24 01:00:00,2020-11-24 01:15:00,Consultation,True
98,E181F837A7F5701CC5537781E94D3537,287ABB19DA8AADBD118443E92685853E,ACB3A881C7CE9ABCAE0CE8C99C86A906,39250431EA8C0D7384AD60C9534AB386,Complete,2020-11-22 10:37:27,2020-11-23 16:00:00,2020-11-24 01:45:00,2020-11-24 02:00:00,Consultation,False


In [30]:
# importing the appointments table to a dataframe for
mdwarehouse_conn = MySqlEngineDW.connect()

doctors_df = pd.read_sql(
    sql="""SELECT * FROM dim_doctor""",
    con=mdwarehouse_conn,
    dtype={
        'age': pd.Int64Dtype()
    }
)

patients_df = pd.read_sql(
    sql="""SELECT * FROM dim_px""",
    con=mdwarehouse_conn,
    dtype={
        'age': pd.Int64Dtype()
    }
)

mdwarehouse_conn.close()

In [31]:
appointments_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 320140 entries, 0 to 320139
Data columns (total 11 columns):
 #   Column      Non-Null Count   Dtype         
---  ------      --------------   -----         
 0   pxID        320140 non-null  object        
 1   clinicID    320140 non-null  object        
 2   doctorID    320140 non-null  object        
 3   apptID      320140 non-null  object        
 4   status      320140 non-null  object        
 5   timequeued  320140 non-null  datetime64[ns]
 6   queuedate   320140 non-null  datetime64[ns]
 7   starttime   203608 non-null  datetime64[ns]
 8   endtime     60152 non-null   datetime64[ns]
 9   type        320140 non-null  object        
 10  virtual     130004 non-null  object        
dtypes: datetime64[ns](4), object(7)
memory usage: 26.9+ MB


In [32]:
appointments_df = pd.merge(
    left=appointments_df,
    right=doctors_df['doctorID'],
    on='doctorID',
    how='inner'
)
appointments_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 320140 entries, 0 to 320139
Data columns (total 11 columns):
 #   Column      Non-Null Count   Dtype         
---  ------      --------------   -----         
 0   pxID        320140 non-null  object        
 1   clinicID    320140 non-null  object        
 2   doctorID    320140 non-null  object        
 3   apptID      320140 non-null  object        
 4   status      320140 non-null  object        
 5   timequeued  320140 non-null  datetime64[ns]
 6   queuedate   320140 non-null  datetime64[ns]
 7   starttime   203608 non-null  datetime64[ns]
 8   endtime     60152 non-null   datetime64[ns]
 9   type        320140 non-null  object        
 10  virtual     130004 non-null  object        
dtypes: datetime64[ns](4), object(7)
memory usage: 26.9+ MB


In [33]:
appointments_df = pd.merge(
    left=appointments_df,
    right=patients_df['pxID'],
    on='pxID',
    how='inner'
)
appointments_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 320125 entries, 0 to 320124
Data columns (total 11 columns):
 #   Column      Non-Null Count   Dtype         
---  ------      --------------   -----         
 0   pxID        320125 non-null  object        
 1   clinicID    320125 non-null  object        
 2   doctorID    320125 non-null  object        
 3   apptID      320125 non-null  object        
 4   status      320125 non-null  object        
 5   timequeued  320125 non-null  datetime64[ns]
 6   queuedate   320125 non-null  datetime64[ns]
 7   starttime   203596 non-null  datetime64[ns]
 8   endtime     60152 non-null   datetime64[ns]
 9   type        320125 non-null  object        
 10  virtual     130004 non-null  object        
dtypes: datetime64[ns](4), object(7)
memory usage: 26.9+ MB


-----

In [34]:
appointments_df.head(100)

,pxID,clinicID,doctorID,apptID,status,timequeued,queuedate,starttime,endtime,type,virtual
0,FE4A5D5A20EC492D2FC691F126A568AB,3B8D83483189887A2F1A39D690463A8F,ACB3A881C7CE9ABCAE0CE8C99C86A906,04BC9218E072BEEFEBEE9C97B78A35C9,Queued,2020-06-22 05:11:38,2020-07-02 16:00:00,2020-07-02 17:00:38,2020-07-02 17:15:38,Consultation,NaN
1,E6BC8E80685AA4A239617F196F12A0C3,3B8D83483189887A2F1A39D690463A8F,ACB3A881C7CE9ABCAE0CE8C99C86A906,F2335AAFB7EE079C7E7306591057C0C9,Queued,2020-06-26 04:51:16,2020-07-12 16:00:00,2020-07-12 17:00:16,2020-07-12 17:15:16,Consultation,NaN
2,E430B3D3AEED550612FFDBD0F2F94112,3B8D83483189887A2F1A39D690463A8F,ACB3A881C7CE9ABCAE0CE8C99C86A906,B2C83D6DE89A88C5CD9EA1FD5FC00602,Queued,2020-07-13 07:00:50,2020-07-19 16:00:00,2020-07-20 05:00:50,2020-07-20 05:15:50,Consultation,NaN
3,E6BC8E80685AA4A239617F196F12A0C3,3B8D83483189887A2F1A39D690463A8F,ACB3A881C7CE9ABCAE0CE8C99C86A906,DA4023B5A8C3F1BD540EA82552F21134,Queued,2020-07-13 06:07:14,2020-07-28 16:00:00,2020-07-29 05:00:13,2020-07-29 05:15:13,Consultation,NaN
4,41E3F930274A7704305EE197F3434877,3B8D83483189887A2F1A39D690463A8F,ACB3A881C7CE9ABCAE0CE8C99C86A906,C805F7992F18E33AB85CEBD572680943,Queued,2020-06-26 05:54:35,2020-09-27 16:00:00,2020-09-27 17:00:35,2020-09-27 17:15:35,Consultation,NaN
...,...,...,...,...,...,...,...,...,...,...,...
95,0173144A9589D9B631D1213CB68CD1F3,287ABB19DA8AADBD118443E92685853E,ACB3A881C7CE9ABCAE0CE8C99C86A906,24BBEF21D8B8E1F63BF9D95D22C52854,Complete,2020-11-19 04:26:14,2020-11-19 16:00:00,2020-11-20 01:30:00,2020-11-20 01:45:00,Consultation,False
96,A4B7F2DEFAB485D3E823EC2E2887CE8B,287ABB19DA8AADBD118443E92685853E,ACB3A881C7CE9ABCAE0CE8C99C86A906,9ECD6AD001663BFF8259BF04932B287E,Complete,2020-11-23 23:37:41,2020-11-23 16:00:00,2020-11-24 03:15:00,2020-11-24 03:30:00,Consultation,True
97,17F29BDC62FF6C1E3EADA946F41BD78C,287ABB19DA8AADBD118443E92685853E,ACB3A881C7CE9ABCAE0CE8C99C86A906,1F61DE829ECDBAA4E09FB099781C47E8,Complete,2020-11-20 02:25:35,2020-11-23 16:00:00,2020-11-24 01:00:00,2020-11-24 01:15:00,Consultation,True
98,E181F837A7F5701CC5537781E94D3537,287ABB19DA8AADBD118443E92685853E,ACB3A881C7CE9ABCAE0CE8C99C86A906,39250431EA8C0D7384AD60C9534AB386,Complete,2020-11-22 10:37:27,2020-11-23 16:00:00,2020-11-24 01:45:00,2020-11-24 02:00:00,Consultation,False


In [39]:
# sending dataframe to sql table
mdwarehouse_conn = MySqlEngineDW.connect()

rows_affected = appointments_df.to_sql(
    name='fact_appointment',
    con=mdwarehouse_conn,
    if_exists='append',
    index=False,
    dtype={
        'pxID': sqlalchemy.types.String(32),
        'clinicID': sqlalchemy.types.String(32),
        'doctorID': sqlalchemy.types.String(32),
        'apptID': sqlalchemy.types.String(32),
        'status':sqlalchemy.types.Enum(StatusEnum),
        'timequeued': sqlalchemy.types.DateTime,
        'queuedate': sqlalchemy.types.DateTime,
        'starttime': sqlalchemy.types.DateTime,
        'endtime': sqlalchemy.types.DateTime,
        'type': sqlalchemy.types.Enum(TypeEnum),
        'virtual': sqlalchemy.types.Boolean
    },
    chunksize=5000,
    method='multi'
)

print('Rows affected:' + str(rows_affected))

mdwarehouse_conn.close()

Rows affected:320125


Standard Protocols will be used to clean data
1. Checking for incorrect data types
2. Check for dupliucate values
3. Check for multiple representations
4. Check for missing and default values
5. Check for inconsistent format

In [ ]:
MySqlEngine.dispose(),
MySqlEngineDW.dispose()